In [9]:
import os
import tensorflow as tf

def get_image_label_lists(images_dir, labels_dir):
    image_paths, class_labels = [], []
    
    for filename in os.listdir(images_dir):
        if filename.endswith('.jpg'):
            img_path = os.path.join(images_dir, filename)
            label_path = os.path.join(labels_dir, filename.replace('.jpg', '.txt'))
            class_id = 0

            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    try:
                        class_id = int(f.readline().split()[0])
                    except:
                        pass

            image_paths.append(img_path)
            class_labels.append(class_id)
    
    return image_paths, class_labels

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224)) / 255.0
    return image, label

def prepare_dataset(image_paths, class_labels, batch_size=32, training=False):
    ds = tf.data.Dataset.from_tensor_slices((image_paths, class_labels))
    ds = ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    if training:
        ds = ds.shuffle(1000)
    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

train_ds = prepare_dataset(*get_image_label_lists("./Dataset/train/images", "./Dataset/train/labels"), training=True)
valid_ds = prepare_dataset(*get_image_label_lists("./Dataset/valid/images", "./Dataset/valid/labels"))
test_ds  = prepare_dataset(*get_image_label_lists("./Dataset/test/images",  "./Dataset/test/labels"))

In [10]:
from collections import Counter
import os

def count_btr_dtr(directory):
    btr_count = 0
    dtr_count = 0
    bv_count = 0
    dv_count = 0
    bt_count = 0
    dt_count = 0
    
    for root, _, files in os.walk(directory):
        for file in files:
            if "BTR" in file:
                btr_count += 1
            elif "DTR" in file:
                dtr_count += 1
            elif "BV" in file:
                bv_count += 1
            elif "DV" in file:
                dv_count += 1
            elif "BT" in file:
                bt_count += 1
            elif "DT" in file:
                dt_count += 1
    
    return Counter({"BTR": btr_count, "DTR": dtr_count, "BV": bv_count, "DV": dv_count, "BT": bt_count, "DT": dt_count})

dataset_dir = "./Dataset"
btr_dtr_counts = count_btr_dtr(dataset_dir)
print(btr_dtr_counts)

Counter({'DTR': 21868, 'BTR': 14778, 'DV': 2078, 'BV': 1402, 'DT': 1056, 'BT': 722})


### Conteo:
- En la carpeta de Train tenemos 21868 imagenes de drones y 14778 imagenes de pajaros

- En la carpeta de valid tenemos 2078 imagenes de drones y 1402 imagenes de pajaros

- En la carpeta de Test tenemos 1056 imagenes de drones y 722 imagenes de pajaros